![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [12]:
-- highest_donation_assignments
WITH cte AS (
	SELECT dt.assignment_id, 
			d.donor_type, 
			ROUND(SUM(dt.amount), 2) AS rounded_total_donation_amount
	FROM donations AS dt
	LEFT JOIN donors AS d
	ON dt.donor_id = d.donor_id
	GROUP BY dt.assignment_id, d.donor_type
	--ORDER BY rounded_total_donation_amount DESC
)
SELECT a.assignment_name,
		a.region,
		c.rounded_total_donation_amount,
		c.donor_type
FROM assignments AS a
INNER JOIN cte AS c
ON a.assignment_id = c.assignment_id
--GROUP BY a.assignment_name, a.region, c.donor_type, c.rounded_total_donation_amount
ORDER BY c.rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [13]:
SELECT dt.assignment_id, 
		d.donor_type, 
		ROUND(SUM(dt.amount), 2) AS rounded_total_donation_amount
FROM donations AS dt
LEFT JOIN donors AS d
ON dt.donor_id = d.donor_id
GROUP BY dt.assignment_id, d.donor_type
ORDER BY rounded_total_donation_amount DESC

,assignment_id,donor_type,rounded_total_donation_amount
0,3033,Individual,3840.66
1,300,Organization,3133.98
2,4114,Organization,2778.57
3,1765,Organization,2626.98
4,268,Individual,2488.69
...,...,...,...
4261,367,Corporate,11.31
4262,1828,Organization,10.84
4263,4958,Individual,10.65
4264,4827,Organization,10.62


In [14]:
-- top_regional_impact_assignments


WITH CTE1 AS (
	SELECT assignment_id, 
		   COUNT(*) AS num_total_donations
	FROM donations
	GROUP BY assignment_id
),
CTE2 AS(
	SELECT a.assignment_id, 
		   a.impact_score,
		   ROW_NUMBER() OVER(PARTITION BY a.region ORDER BY a.impact_score DESC) AS rank
	FROM assignments AS a
	RIGHT JOIN CTE1 AS c
	ON c.assignment_id = a.assignment_id
	GROUP BY a.assignment_id, a.impact_score
)

SELECT a.assignment_name, 
		a.region, 
		a.impact_score, 
		c1.num_total_donations
FROM assignments AS a
RIGHT JOIN CTE2 AS c2
ON c2.assignment_id = a.assignment_id
RIGHT JOIN CTE1 AS c1
ON c1.assignment_id = c2.assignment_id
WHERE c2.rank = 1
ORDER BY a.region 



,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2


In [15]:
SELECT assignment_id, 
		COUNT(*) AS num_total_donations
FROM donations
GROUP BY assignment_id


,assignment_id,num_total_donations
0,1489,1
1,1269,1
2,2335,3
3,4790,2
4,51,1
...,...,...
3183,1715,2
3184,55,1
3185,4827,1
3186,790,1


In [16]:
SELECT assignment_id, 
		impact_score,
		ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS rank
FROM assignments
GROUP BY assignment_id, impact_score

,assignment_id,impact_score,rank
0,316,10.00,1
1,1514,9.99,2
2,3624,9.98,3
3,3196,9.97,4
4,2281,9.95,5
...,...,...,...
4995,3778,1.05,1250
4996,1150,1.05,1251
4997,2518,1.04,1252
4998,1033,1.02,1253
